In [12]:
# Import Dependencies
import pandas as pd
import json
import glob
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [13]:
from google.colab import drive

%cd /content/drive/MyDrive/ColabNotebooks/MSDO_Predictor


/content/drive/MyDrive/ColabNotebooks/MSDO_Predictor


In [14]:
# DATA PREPARATION - New JSON

%cd /content/drive/MyDrive/ColabNotebooks/MSDO_Predictor

# DATA PREPARATION - New JSON
new_df_combined = pd.read_json('ALLsemesters.json')
new_df_combined.drop('toeflScore', axis=1, inplace=True)

# Extract institution, degree, and GPA from 'education' field
new_df_combined['institution'] = new_df_combined['education'].apply(lambda x: x['institution'] if isinstance(x, dict) and 'institution' in x else None)
new_df_combined['degree'] = new_df_combined['education'].apply(lambda x: x['degree'] if isinstance(x, dict) and 'degree' in x else None)
new_df_combined['gpa'] = new_df_combined['education'].apply(lambda x: x['gpa'] if isinstance(x, dict) and 'gpa' in x else None)

new_df_combined['gre_quantitative'] = new_df_combined['gre'].apply(lambda x: x['quantitative'] if isinstance(x, dict) and 'quantitative' in x else None)
new_df_combined['gre_verbal'] = new_df_combined['gre'].apply(lambda x: x['verbal'] if isinstance(x, dict) and 'verbal' in x else None)
new_df_combined['gre_writing'] = new_df_combined['gre'].apply(lambda x: x['writing'] if isinstance(x, dict) and 'writing' in x else None)


# Drop the several columns
new_df_combined = new_df_combined.drop('education', axis=1)
new_df_combined = new_df_combined.drop('applicationDate', axis=1)
new_df_combined = new_df_combined.drop('decisionDate', axis=1)
new_df_combined = new_df_combined.drop('gre', axis=1)

new_df_combined

/content/drive/MyDrive/ColabNotebooks/MSDO_Predictor


,status,recommendations,experience,statementOfPurpose,comments,moocs,relevantCreditCourses,institution,degree,gpa,gre_quantitative,gre_verbal,gre_writing
0,Accepted,0,"Unrelated(Active Duty Air Force), open source ...",Y,"All A's for prereq's, including the ones on th...",NaN,NaN,Big State School,MS in Software Engineering,4.00,NaN,NaN,NaN
1,Accepted,1,"3 years as machine learning engineer, 3 years ...",Y,I also have publications and thesis. Generally...,NaN,NaN,University of Minnesota,MS in Computer Science,3.70,NaN,NaN,NaN
2,Rejected,1,"3 years, manufacturing, Python & SQL",Y,"Got rejected, but I've already accepted an adm...",NaN,NaN,University of Illinois,BS in Mechanical Engineering,3.75,NaN,NaN,NaN
3,Accepted,3,5 years in research in academia which involved...,Y,Nervous and will need to do a lot of studying ...,NaN,NaN,None,BS in Health Sciences,NaN,NaN,NaN,NaN
4,Accepted,3,NaN,Y,My Weakness - I know my 3.92 GPA seems like a ...,NaN,NaN,UT Austin,BS in Data Science,3.92,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Accepted,2,Manufacturing Operations and Quality 8 yrs,Talked about experience working on Stats and r...,NaN,Coursera ML Mathematics and Python,NaN,None,BS in Mechanical Engineering,3.50,NaN,NaN,NaN
68,Accepted,3,"Community college professor (tenured, 8 years)...",Explained the big project I wanted to work on ...,NaN,none,NaN,UCSD,MS in Materials Science,3.80,NaN,NaN,NaN
69,Accepted,0,"Not much, but I just accepted a position as an...",Explained how I am passionate about building a...,NaN,1 - Deep Learning with PyTorch from IBM,NaN,UT Austin,MA in Economics,3.60,160.0,168.0,4.5
70,Accepted,2,"15 Years, Cloud, DevOps, Python, ML",Explained how my previous experience is relate...,NaN,Andrew NG's ML Specialization,NaN,None,BS in Mechanical Engineering,3.40,NaN,NaN,NaN


In [15]:
import re

def calculate_total_experience(text):
    if isinstance(text, str):
        total_years = 0
        pattern_years = r"(\d+)\s*(year|yr|years)"
        pattern_months = r"(\d+)\s*(month|months)"

        # Find mentions of years
        matches_years = re.findall(pattern_years, text)
        for match in matches_years:
            years = int(match[0])
            total_years += years

        # Find mentions of months and convert to years
        matches_months = re.findall(pattern_months, text)
        for match in matches_months:
            months = int(match[0])
            years = months / 12
            total_years += years

        return total_years

    return 0

# Apply the calculate_total_experience function to the 'experience' column
new_df_combined['total_experience'] = new_df_combined['experience'].apply(calculate_total_experience)
new_df_combined = new_df_combined.drop('experience', axis=1)

new_df_combined

,status,recommendations,statementOfPurpose,comments,moocs,relevantCreditCourses,institution,degree,gpa,gre_quantitative,gre_verbal,gre_writing,total_experience
0,Accepted,0,Y,"All A's for prereq's, including the ones on th...",NaN,NaN,Big State School,MS in Software Engineering,4.00,NaN,NaN,NaN,0.0
1,Accepted,1,Y,I also have publications and thesis. Generally...,NaN,NaN,University of Minnesota,MS in Computer Science,3.70,NaN,NaN,NaN,6.0
2,Rejected,1,Y,"Got rejected, but I've already accepted an adm...",NaN,NaN,University of Illinois,BS in Mechanical Engineering,3.75,NaN,NaN,NaN,3.0
3,Accepted,3,Y,Nervous and will need to do a lot of studying ...,NaN,NaN,None,BS in Health Sciences,NaN,NaN,NaN,NaN,5.0
4,Accepted,3,Y,My Weakness - I know my 3.92 GPA seems like a ...,NaN,NaN,UT Austin,BS in Data Science,3.92,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Accepted,2,Talked about experience working on Stats and r...,NaN,Coursera ML Mathematics and Python,NaN,None,BS in Mechanical Engineering,3.50,NaN,NaN,NaN,8.0
68,Accepted,3,Explained the big project I wanted to work on ...,NaN,none,NaN,UCSD,MS in Materials Science,3.80,NaN,NaN,NaN,8.0
69,Accepted,0,Explained how I am passionate about building a...,NaN,1 - Deep Learning with PyTorch from IBM,NaN,UT Austin,MA in Economics,3.60,160.0,168.0,4.5,0.0
70,Accepted,2,Explained how my previous experience is relate...,NaN,Andrew NG's ML Specialization,NaN,None,BS in Mechanical Engineering,3.40,NaN,NaN,NaN,0.0


In [16]:
listcolumn = ['moocs',	'relevantCreditCourses', 'gre_quantitative',	'gre_verbal',	'gre_writing']
new_df_combined = new_df_combined.drop(listcolumn, axis=1)

new_df_combined

,status,recommendations,statementOfPurpose,comments,institution,degree,gpa,total_experience
0,Accepted,0,Y,"All A's for prereq's, including the ones on th...",Big State School,MS in Software Engineering,4.00,0.0
1,Accepted,1,Y,I also have publications and thesis. Generally...,University of Minnesota,MS in Computer Science,3.70,6.0
2,Rejected,1,Y,"Got rejected, but I've already accepted an adm...",University of Illinois,BS in Mechanical Engineering,3.75,3.0
3,Accepted,3,Y,Nervous and will need to do a lot of studying ...,None,BS in Health Sciences,NaN,5.0
4,Accepted,3,Y,My Weakness - I know my 3.92 GPA seems like a ...,UT Austin,BS in Data Science,3.92,0.0
...,...,...,...,...,...,...,...,...
67,Accepted,2,Talked about experience working on Stats and r...,NaN,None,BS in Mechanical Engineering,3.50,8.0
68,Accepted,3,Explained the big project I wanted to work on ...,NaN,UCSD,MS in Materials Science,3.80,8.0
69,Accepted,0,Explained how I am passionate about building a...,NaN,UT Austin,MA in Economics,3.60,0.0
70,Accepted,2,Explained how my previous experience is relate...,NaN,None,BS in Mechanical Engineering,3.40,0.0


In [17]:
new_df_combined

,status,recommendations,statementOfPurpose,comments,institution,degree,gpa,total_experience
0,Accepted,0,Y,"All A's for prereq's, including the ones on th...",Big State School,MS in Software Engineering,4.00,0.0
1,Accepted,1,Y,I also have publications and thesis. Generally...,University of Minnesota,MS in Computer Science,3.70,6.0
2,Rejected,1,Y,"Got rejected, but I've already accepted an adm...",University of Illinois,BS in Mechanical Engineering,3.75,3.0
3,Accepted,3,Y,Nervous and will need to do a lot of studying ...,None,BS in Health Sciences,NaN,5.0
4,Accepted,3,Y,My Weakness - I know my 3.92 GPA seems like a ...,UT Austin,BS in Data Science,3.92,0.0
...,...,...,...,...,...,...,...,...
67,Accepted,2,Talked about experience working on Stats and r...,NaN,None,BS in Mechanical Engineering,3.50,8.0
68,Accepted,3,Explained the big project I wanted to work on ...,NaN,UCSD,MS in Materials Science,3.80,8.0
69,Accepted,0,Explained how I am passionate about building a...,NaN,UT Austin,MA in Economics,3.60,0.0
70,Accepted,2,Explained how my previous experience is relate...,NaN,None,BS in Mechanical Engineering,3.40,0.0
